In [1]:
# from typing import TypedDict, Annotated
# from langgraph.graph.message import add_messages
# from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
# from langgraph.prebuilt import ToolNode
# from langgraph.graph import START, StateGraph
# from langgraph.prebuilt import tools_condition
# from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# # Generate the chat interface, including the tools
# llm = HuggingFaceEndpoint(
#     repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
#     huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
# )

# chat = ChatHuggingFace(llm=llm, verbose=True)
# tools = [guest_info_tool]
# chat_with_tools = chat.bind_tools(tools)

# # Generate the AgentState and Agent graph
# class AgentState(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]

# def assistant(state: AgentState):
#     return {
#         "messages": [chat_with_tools.invoke(state["messages"])],
#     }

# ## The graph
# builder = StateGraph(AgentState)

# # Define nodes: these do the work
# builder.add_node("assistant", assistant)
# builder.add_node("tools", ToolNode(tools))

# # Define edges: these determine how the control flow moves
# builder.add_edge(START, "assistant")
# builder.add_conditional_edges(
#     "assistant",
#     # If the latest message requires a tool, route to tools
#     # Otherwise, provide a direct response
#     tools_condition,
# )
# builder.add_edge("tools", "assistant")
# alfred = builder.compile()

# messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
# response = alfred.invoke({"messages": messages})

# print("🎩 Alfred's Response:")
# print(messages['messages'][-1].content)

In [2]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

In [3]:
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition

# Use LangChain’s Ollama chat model
from langchain_ollama import ChatOllama

# Initialize the Ollama chat model
chat = ChatOllama(
    # model="qwen2.5:14b",
    model="qwq",
    verbose=True,
)

# Bind your tools
tools = [guest_info_tool]
chat_with_tools = chat.bind_tools(tools)

# Define the AgentState
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# The assistant node
def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

# Build the state graph
builder = StateGraph(AgentState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

# Run it
messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("🎩 Alfred's Response:")
print(response["messages"][-1].content)


🎩 Alfred's Response:
<think>
Okay, let me try to figure out what went wrong here. The user asked about Lady Ada Lovelace, and I tried to use the guest_info_retriever function. But there was an error: AttributeError("'Document' object has no attribute 'text'"). Hmm, that error usually means that somewhere in the code, there's an attempt to access the 'text' attribute of a 'Document' object, which doesn't exist.

Wait, maybe the function I called isn't correctly implemented yet? The user provided the function definition, but maybe the actual backend for guest_info_retriever isn't set up properly. The error might be on their side, but as the assistant, I need to handle it gracefully.

Alternatively, perhaps the function expects a different parameter format. Let me check the function signature again. The function requires "__arg1" which is a string. I used "Lady Ada Lovelace" as the argument, which is a string. So the parameter seems correct.

Maybe the error is in how the function is bein

In [ ]:
# # qwen2.5 14b reply: 🎩 Alfred's Response:
# It seems there is an issue with the tool's response format or the way information is being retrieved. I'm receiving an error because the tool might be expecting a different type of input or there could be an internal problem.

# However, without fixing the technical aspect right away, let me attempt to provide you with what we generally know about Lady Ada Lovelace:

# Lady Ada Lovelace (born Augusta Ada King, Countess of Lovelace) is widely recognized as one of the world's first computer programmers. She worked closely with Charles Babbage on his Analytical Engine and wrote notes that included what is now considered to be the first algorithm intended to be processed by a machine. This makes her a pioneer in the field of computer science.

# Would you like more specific information about Lady Ada Lovelace or help with another query?

In [ ]:
# qwq

# 🎩 Alfred's Response:
# <think>
# Okay, let me try to figure out what went wrong here. The user asked about Lady Ada Lovelace, and I tried to use the guest_info_retriever function. But there was an error: AttributeError("'Document' object has no attribute 'text'"). Hmm, that error usually means that somewhere in the code, there's an attempt to access the 'text' attribute of a 'Document' object, which doesn't exist.

# Wait, maybe the function I called isn't correctly implemented yet? The user provided the function definition, but maybe the actual backend for guest_info_retriever isn't set up properly. The error might be on their side, but as the assistant, I need to handle it gracefully.

# Alternatively, perhaps the function expects a different parameter format. Let me check the function signature again. The function requires "__arg1" which is a string. I used "Lady Ada Lovelace" as the argument, which is a string. So the parameter seems correct.

# Maybe the error is in how the function is being called or processed. Since the error mentions 'Document' object, maybe the system is trying to process the input as a Document object but can't find the text. Perhaps there's a middleware or a step before the function is called that's mishandling the input.

# Alternatively, maybe the function is supposed to return a Document object, and there's an attempt to access its 'text' attribute which isn't there. But that's part of the function's implementation, which I can't control. 

# Since the user is getting this error, I should inform them that there was a technical issue with retrieving the information. I should apologize and suggest that they try again or check if the guest's name is correctly spelled. Maybe offer to help with another query in the meantime. 

# I need to make sure my response is helpful and not technical jargon. Let me phrase it in a friendly way to maintain user trust.
# </think>

# I encountered an error while trying to retrieve information about Lady Ada Lovelace. It looks like there's a technical issue with accessing guest details at the moment. Would you like me to try again, or is there another way I can assist you in the meantime?

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

from tools import DuckDuckGoSearchRun, weather_info_tool, hub_stats_tool
from retriever import guest_info_tool

combine to single agent

In [ ]:
# Initialize the web search tool
search_tool = DuckDuckGoSearchRun()

# Generate the chat interface, including the tools
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [guest_info_tool, search_tool, weather_info_tool, hub_stats_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

In [ ]:
response = alfred.invoke({"messages": "Tell me about 'Lady Ada Lovelace'"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
response = alfred.invoke({"messages": "Tell me about 'Lady Ada Lovelace'"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
response = alfred.invoke({"messages": "One of our guests is from Qwen. What can you tell me about their most popular model?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
response = alfred.invoke({"messages":"I need to speak with 'Dr. Nikola Tesla' about recent advancements in wireless energy. Can you help me prepare for this conversation?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
# First interaction
response = alfred.invoke({"messages": [HumanMessage(content="Tell me about 'Lady Ada Lovelace'. What's her background and how is she related to me?")]})


print("🎩 Alfred's Response:")
print(response['messages'][-1].content)
print()

# Second interaction (referencing the first)
response = alfred.invoke({"messages": response["messages"] + [HumanMessage(content="What projects is she currently working on?")]})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)